# Catboost Hyperparameter Sweep 20210918
Largely after https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu

# Setup

In [1]:
# two manual flags (ex-config)
colab = False
gpu_available = False

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random
import multiprocessing
import pickle

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"sweep_xgboost_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# handle Google Colab-specific library installation/updating
if colab:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
    !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    !pip install --upgrade xgboost

    # upgrade sklearn
    !pip install --upgrade scikit-learn

    !pip install category_encoders
    !pip install catboost

    # lighgbm gpu compatible
    !git clone --recursive https://github.com/Microsoft/LightGBM
    ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
    
    # this part is from https://github.com/rapidsai/gputreeshap/issues/24
    !pip install cmake --upgrade
    # !pip install sklearn --upgrade
    !git clone --recursive https://github.com/dmlc/xgboost
    %cd /content/xgboost
    !mkdir build
    %cd build
    !cmake .. -DUSE_CUDA=ON
    !make -j4
    %cd /content/xgboost/python-package
    !python setup.py install --use-cuda --use-nccl
    !/opt/bin/nvidia-smi
    !pip install shap
    

Now, non-stdlib imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from optuna.integration.wandb import WeightsAndBiasesCallback
# from wandb.xgboost import wandb_callback
# from wandb.lightgbm import wandb_callback
# from sklearn.impute import KNNImputer, StandardImputer
# import timm

import seaborn as sns

# from catboost import CatBoostClassifier
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


from optuna.samplers import TPESampler
import optuna
import catboost
from sklearn.utils import resample
import sklearn.metrics

Now, datapath setup

In [6]:
# # This is the code for reading the train.csv and converting it to a .feather file
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

In [7]:
if colab:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/sep2021/')
    
else:
    # if on local machine
    datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')    
    


In [8]:

n_trials = int(1000)
SEED = 42

In [9]:
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(SEED)

## Ex-Model Config

In [10]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
# in the sweep version, this includes both ex-model parameters and defaults for model parameters
exmodel_config = {
    # model config
    "library": 'xgboost',
#     "model": XGBClassifier,
#     "n_estimators": 100, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "test_size": 0.2,
#     "reg_lambda": None, 
    "scaler": "sklearn.preprocessing.StandardScaler()", # TODO: experiment with others (but imputation may be slow)
    "scale_b4_impute": False,
    "imputer": "sklearn.impute.SimpleImputer(strategy='median', add_indicator=True)",
    "knn_imputer_n_neighbors": None, # None if a different imputer is used
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': SEED,
    'optuna': True,
    'optuna_trials': 1000,
#     'subsample': 1,
#     'cross_val_strategy': None, # None for holdout, or the relevant sklearn class
#     'kfolds': 1, # if 1, that means just doing holdout
#     'test_size': 0.2,
    # these are XGBoost default (my choice) params 
#     "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
#     "booster": 'gbtree', # dart may be marginally better, but will opt for this quicker approach as a default
#     "n_estimators": 200, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "n_jobs": -1,
#     "verbosity": 1,
#     "subsample": 1,
#     'features_created': False,
#     'feature_creator': None,
}

wandb_kwargs = {
    # wandb config
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'project': '202109_Kaggle_tabular_playground',
    'tags': ['sweep'],
    'notes': "Sweep for XGBoost using Optuna",
    'config': exmodel_config,
}

## Data Setup

**TODO** Write some conditional logic here to automate it -- possibly as part of a sklearn.*pipeline

In [11]:
X_source = 'X_NaNcounts_SummaryStats_imputed-Median-wIndicators-StandardScaled.feather'
X_train = pd.read_feather(datapath/X_source) 
y_train = load(datapath/'y.joblib')    
# X.index.name = 'id'
# y.index.name = 'id'
X = np.array(X_train)
y = np.array(y_train)

del X_train, y_train

In [12]:
exmodel_config['feature_count'] = X.shape[1]
exmodel_config['feature_generator'] = "Summary statistics"
exmodel_config['X_source'] = X_source

# Experiment setup

In [13]:
# wandb_kwargs = {
#     # wandb config:
#     'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
#     'project': '202109_Kaggle_tabular_playground',
#     'tags': ['sweep'],
#     'notes': "Sweep for CatBoost using Optuna",
#     'config': exmodel_config,
# }

In [14]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial):
    # split the (original Kaggle training) data into partitions
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=int(SEED), shuffle=True)
    # create wrappers for the training and validation partitions
#     train_pool = catboost.Pool(train_x, train_y)
#     test_pool = catboost.Pool(test_x, test_y)
    
    # experimental parameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1200),
        'max_depth' : trial.suggest_int('depth', 2, 7),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.4),               
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.001, 5),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1, 7),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1),
#         'num_leaves': trial.suggest_int('num_leaves', )
    }    
        
    tree_method = 'gpu_hist' if (gpu_available and colab) else 'auto' 
        
    
    # instantiate the model, with some parameters locked in, and experimnental ones passed via splat 
    model = XGBClassifier(
        objective='binary:logistic',
        verbosity=1,
        tree_method=tree_method,
        random_state=SEED,
        n_jobs=-1,
        **params
    )        
#     wandb.init(wandb_kwargs)
    # fit the model on the training set -- note the use of the catboost.Pool instance
    model.fit(X_train, y_train)
    # generate predictions
    preds = model.predict_proba(X_valid)[:,1]
    # rounds to the nearest integer, and the nearest even in case of _.5s
#     pred_labels = np.rint(preds)
    # calls sklearn.utils.resample... not entirely sure why
#     y_pred_boot = resample(pred_labels, n_samples = len(train_y))
    # Evaluation
#     ROC_AUC_Score = roc_auc_score(train_y, y_pred_boot)
    valid_auc = roc_auc_score(y_valid, preds)
    print('ROC AUC Score of XGBoost =', valid_auc)
    wandb.log({'valid_auc': valid_auc,
#                'catboost_params': str(model.get_all_params())
              })
#     wandb.finish()
    return valid_auc

In [15]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

<ipython-input-15-69ea9289a2cf>:1: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [16]:
study = optuna.create_study(direction = "maximize", 
                            sampler = TPESampler(seed=int(SEED)), 
                            study_name='xgboost_20210919')
# setting n_jobs=1 bc wandb apparently requires this
study.optimize(objective, n_trials = n_trials, callbacks = [wandbc]) #n_jobs = multiprocessing.cpu_count())

[I 2021-09-20 13:28:28,534] A new study created in memory with name: xgboost_20210919
/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:28:33] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2021-09-20 13:37:51,697] Trial 0 finished with value: 0.8118794328459813 and parameters: {'n_estimators': 512, 'depth': 7, 'learning_rate': 0.0802956743641955, 'reg_alpha': 0.16383993835282307, 'reg_lambda': 1.3547246940681168, 'subsample': 0.5779972601681014}. Best is trial 0 with value: 0.8118794328459813.
wandb: WARNING Step must only increase in log calls.  Step 0 < 1; dropping {'n_estimators': 512, 'depth': 7, 'learning_rate': 0.0802956743641955, 'reg_alpha': 0.16383993835282307, 'reg_lambda': 1.3547246940681168, 'subsample': 0.5779972601681014, 'value': 0.8118794328459813}.


ROC AUC Score of XGBoost = 0.8118794328459813


/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[I 2021-09-21 08:42:51,361] Trial 100 finished with value: 0.8148505806059572 and parameters: {'n_estimators': 1132, 'depth': 6, 'learning_rate': 0.03349111825997133, 'reg_alpha': 2.9742522652968173, 'reg_lambda': 2.4920359287670286, 'subsample': 0.9022174040798057}. Best is trial 77 with value: 0.8151879038955687.
wandb: WARNING Step must only increase in log calls.  Step 100 < 101; dropping {'n_estimators': 1132, 'depth': 6, 'learning_rate': 0.03349111825997133, 'reg_alpha': 2.9742522652968173, 'reg_lambda': 2.4920359287670286, 'subsample': 0.9022174040798057, 'value': 0.8148505806059572}.


ROC AUC Score of XGBoost = 0.8148505806059572


/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:42:56] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [17]:
dump(study, filename='optuna_xgboost_study_20210920.joblib')
dump(study.best_trial.params, filename='optuna_xgboost_best_20210920.joblib')

# pickle.dump(study.best_trial.params, open('CatBoost_Hyperparameter.pickle', 'wb'))
# print('CatBoost Hyperparameter:', study.best_trial.params)

['optuna_xgboost_best_20210920.joblib']

In [18]:
study.best_trial.params

{'n_estimators': 1119,
 'depth': 6,
 'learning_rate': 0.04123392555159452,
 'reg_alpha': 4.511876752318655,
 'reg_lambda': 4.074347238862406,
 'subsample': 0.8408586950521992}